In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('data.csv')

In [5]:
df.head()

,tokenize_text
0,бухгалтер энди дюфрейн обвинить убийство собст...
1,пол эджкомба начальник блок смертник тюрьма хо...
2,лицо главное герой форрест гампа слабоумный бе...
3,фильм рассказывать реальный история загадочный...
4,пострадать результат несчастный случай богатый...


In [48]:
russian_stopwords = stopwords.words("russian") 

In [50]:
russian_stopwords.extend(['т.д.', 'т', 'д', 'это','который','свой','своём','всем','всё','её','оба','ещё','должный']) 

In [15]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))

In [17]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['tokenize_text'])

In [19]:
tfidf_matrix.shape

(250, 911)

## Тематическое моделирование

In [23]:
from sklearn.decomposition import TruncatedSVD

In [25]:
# создание модели LSA
lsa_model = TruncatedSVD(n_components=5, random_state=0)
lsa_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(lsa_model.components_):
    print(f"Topic {i}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

Topic 0: жизнь, год, человек, друг, мочь, весь, мир, время, судьба, стать
Topic 1: друг, шерлок холмс, холмс, шерлок, ватсон, доктор, доктор ватсон, схватка, друг друг, лондон
Topic 2: друг, друг друг, девушка, жить, удивительный, стать, однажды, любовь, странный, жизнь
Topic 3: время, марти, век, год, машина, история, попадать, машина время, приключение, герой
Topic 4: друг, сын, война, отец, новый, любовь, капитан, бой, джек, время


In [28]:
from sklearn.decomposition import NMF

In [30]:
# создание модели NMF
nmf_model = NMF(n_components=5, random_state=0)
nmf_model.fit(tfidf_matrix)

# вывод топ слов для каждой темы
for i, topic in enumerate(nmf_model.components_):
    print(f"Topic {i}: {', '.join([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])}")

Topic 0: ребёнок, мир, мочь, отец, человек, весь, девушка, жизнь, дело, жить
Topic 1: холмс, шерлок, шерлок холмс, ватсон, доктор, доктор ватсон, схватка, убийца, сыщик, угроза
Topic 2: друг, друг друг, стать, любовь, сильный, год, новый, девушка, однажды, удивительный
Topic 3: жизнь, год, время, герой, фильм, век, попадать, каждый, мечта, машина
Topic 4: война, бой, сын, мировой, судьба, последний, мировой война, человек, время, второй мировой


## Кластеризация

In [245]:
num_clusters = 5

# Метод к-средних - KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters, random_state=0)

In [247]:
km.fit(tfidf_matrix)

KMeans(n_clusters=5, random_state=0)

In [249]:
clusterkm = km.labels_.tolist()
df['cluster']= clusterkm

In [251]:
df.head()

,tokenize_text,cluster
0,бухгалтер энди дюфрейн обвинить убийство собст...,0
1,пол эджкомба начальник блок смертник тюрьма хо...,1
2,лицо главное герой форрест гампа слабоумный бе...,0
3,фильм рассказывать реальный история загадочный...,4
4,пострадать результат несчастный случай богатый...,1


In [253]:
df['cluster'].value_counts()

cluster
3    63
4    51
2    49
1    45
0    42
Name: count, dtype: int64

In [516]:
df[df['cluster']==3]

,tokenize_text,cluster
11,судьба сводить герой картина больница врач вын...,3
18,последний часть трилогия кольцо всевластие гер...,3
19,великий римский империя военачальник равный ге...,3
25,фрэнк эбегнейл успеть поработать врач адвокат ...,3
28,эскадрилья стать петь капитан титаренко подбир...,3
...,...,...
240,узнать умереть бывший невеста шахтёр павел зуб...,3
241,погибнуть 1944 год весь восемнадцать человек к...,3
244,1992 год абхазия древний эстонский село распол...,3
247,флера шестнадцатилетний мальчишка откопать сре...,3


## Классификация

In [256]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['tokenize_text'], df['cluster'], 
                                                      test_size=0.3, 
                                                      random_state=0)

In [412]:
len(X_train)

175

In [414]:
len(X_test)

75

In [258]:
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [260]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,3), stop_words=russian_stopwords)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## LogisticRegression

In [263]:
model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)

LogisticRegression()

In [265]:
y_pred = model_lr.predict(X_test_tfidf)

In [267]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.50      0.12      0.20         8
           2       1.00      0.26      0.42        19
           3       0.27      1.00      0.42        17
           4       0.75      0.21      0.33        14

    accuracy                           0.35        75
   macro avg       0.50      0.32      0.27        75
weighted avg       0.51      0.35      0.28        75



In [269]:
X_test_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [273]:
y_pred

array([3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 1,
       1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3,
       3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

## RandomForestClassifier

In [276]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [278]:
y_pred = model_rf.predict(X_test_tfidf)

In [282]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.18      0.29        17
           1       0.23      0.75      0.35         8
           2       1.00      0.47      0.64        19
           3       0.29      0.53      0.38        17
           4       0.40      0.14      0.21        14

    accuracy                           0.39        75
   macro avg       0.53      0.41      0.37        75
weighted avg       0.59      0.39      0.39        75



## KNeighborsClassifier

In [286]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train_tfidf, y_train)

KNeighborsClassifier()

In [288]:
y_pred = model_knn.predict(X_test_tfidf)

In [290]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.33      0.24      0.28        17
           1       0.33      0.38      0.35         8
           2       0.64      0.74      0.68        19
           3       0.33      0.41      0.37        17
           4       0.45      0.36      0.40        14

    accuracy                           0.44        75
   macro avg       0.42      0.42      0.42        75
weighted avg       0.43      0.44      0.43        75



## Сохранение моделей

In [306]:
import pickle

with open('model_knn1.pkl', 'wb') as f:
    pickle.dump(model_knn, f)

In [374]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [308]:
df.to_csv('data_movies.csv')

t1 = "Бухгалтер Энди Дюфрейн обвинён в\xa0убийстве собственной жены и\xa0её любовника. Оказавшись в\xa0тюрьме под\xa0названием Шоушенк; он\xa0сталкивается с\xa0жестокостью и\xa0беззаконием; царящими по\xa0обе стороны решётки. Каждый; кто\xa0попадает в\xa0эти стены; становится их\xa0рабом до\xa0конца жизни. Но\xa0Энди; обладающий живым умом и\xa0доброй душой; находит подход как\xa0к заключённым; так\xa0и к\xa0охранникам; добиваясь их\xa0особого к\xa0себе расположения."

In [312]:
t2 = "собственный жена любовник оказаться тюрьма название шоушенк сталкиваться жестокость беззаконие царить сторона решётка каждый попадать стена становиться раб конец жизнь обладать живой ум добрый душа находить подход заключить охранник добиваться особый расположение"

t3 = "21 июня 1941 года. Молодой лейтенант Коля Плужников, получив назначение на постоянное место службы, приезжает в Брест. Переполненные залы ожидания вокзала и толпа увешанных багажом людей не настораживают охваченного радостными надеждами юношу. Коля спешит к месту расположения своей части — в Брестскую крепость… Солдата не успевают зачислить в личный состав военнослужащих, а в четыре утра раздаются артиллерийские разрывы — началась война. Он не был зачислен в ряды военнослужащих Брестской крепости, но всё равно принял участие в бою."

t4 = "Российский ракетный подводный крейсер специального назначения бесследно исчезает во время секретной миссии в Гренландском море. На его поиски отправляется экипаж под командованием Виктора Воронина, чей старший брат командовал пропавшей субмариной. Перед моряками стоит задача любыми силами найти подлодку и не допустить, чтобы новое секретное оружие попало в руки врага. В это же время из-за геологических исследований на полярной станции в северных водах пробуждается Кракен — гигантское чудовище, способное сливаться с подводной тьмой, обладающее высоким интеллектом. Именно с ним предстоит столкнуться морякам, разыскивающим пропавшую подводную лодку."

In [14]:
import pickle
import joblib
from catboost import CatBoostClassifier
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
t1 = "Бухгалтер Энди Дюфрейн обвинён в\xa0убийстве собственной жены и\xa0её любовника. Оказавшись в\xa0тюрьме под\xa0названием Шоушенк; он\xa0сталкивается с\xa0жестокостью и\xa0беззаконием; царящими по\xa0обе стороны решётки. Каждый; кто\xa0попадает в\xa0эти стены; становится их\xa0рабом до\xa0конца жизни. Но\xa0Энди; обладающий живым умом и\xa0доброй душой; находит подход как\xa0к заключённым; так\xa0и к\xa0охранникам; добиваясь их\xa0особого к\xa0себе расположения."

In [16]:
with open('model_knn1.pkl', 'rb') as file:
    model = pickle.load(file)

with open('vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

In [18]:
def fun_punctuation_text(text):
    text = text.lower()
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    text = ''.join([i if not i.isdigit() else '' for i in text])
    text = ''.join([i if i.isalpha() else ' ' for i in text])
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub('[a-z]', '', text, flags=re.I)
    st = '❯\xa0'
    text = ''.join([ch if ch not in st else ' ' for ch in text])
    return text

In [20]:
def fun_lemmatizing_text(text):
    tokens = word_tokenize(text)
    res = list()
    for word in tokens:
        p = pymorphy3.MorphAnalyzer(lang='ru').parse(word)[0]
        res.append(p.normal_form)  
    text = " ".join(res)
    return text

In [22]:
def fun_tokenize(text):
    t = word_tokenize(text)
    tokens = [token for token in t if token not in russian_stopwords]
    text = " ".join(tokens)
    return text

In [30]:
t1 = fun_punctuation_text(t1)

In [32]:
t1 = fun_lemmatizing_text(t1)
t1

'бухгалтер энди дюфрейн обвинить в убийство собственный жена и её любовник оказаться в тюрьма под название шоушенк он сталкиваться с жестокость и беззаконие царить по оба сторона решётка каждый кто попадать в этот стена становиться они раб до конец жизнь но энди обладать живой ум и добрый душа находить подход как к заключить так и к охранник добиваться они особый к себя расположение'

In [52]:
t1 = fun_tokenize(t1)
t1

'бухгалтер энди дюфрейн обвинить убийство собственный жена любовник оказаться тюрьма название шоушенк сталкиваться жестокость беззаконие царить сторона решётка каждый попадать стена становиться раб конец жизнь энди обладать живой ум добрый душа находить подход заключить охранник добиваться особый расположение'

In [54]:
text_vectorized = vectorizer.transform([t1])

In [56]:
text_vectorized.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [58]:
prediction = model.predict(text_vectorized)  
probabilities = model.predict_proba(text_vectorized)

In [60]:
print(f"Класс: {prediction[0]}")
print(f"Вероятности: {probabilities}")

Класс: 0
Вероятности: [[0.8 0.2 0.  0.  0. ]]


In [63]:
t3 = "21 июня 1941 года. Молодой лейтенант Коля Плужников, получив назначение на постоянное место службы, приезжает в Брест. Переполненные залы ожидания вокзала и толпа увешанных багажом людей не настораживают охваченного радостными надеждами юношу. Коля спешит к месту расположения своей части — в Брестскую крепость… Солдата не успевают зачислить в личный состав военнослужащих, а в четыре утра раздаются артиллерийские разрывы — началась война. Он не был зачислен в ряды военнослужащих Брестской крепости, но всё равно принял участие в бою."

In [65]:
t3 = fun_punctuation_text(t3)

In [67]:
t3 = fun_lemmatizing_text(t3)

In [68]:
t3 = fun_tokenize(t3)

In [69]:
text_vectorized = vectorizer.transform([t3])

In [70]:
prediction = model.predict(text_vectorized)  
probabilities = model.predict_proba(text_vectorized)

In [71]:
print(f"Класс: {prediction[0]}")
print(f"Вероятности: {probabilities}")

Класс: 3
Вероятности: [[0.2 0.2 0.2 0.4 0. ]]
